In [28]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
from datasets import Dataset
from transformers import AutoProcessor
import torch


In [42]:
cwd = str(Path.cwd())
filepath = cwd + "/datasets/train-00000-of-00738-80a58552f2fb3344.parquet"

model = cwd + "/microsoft/phi-3.5-vision-instruct"
processor = AutoProcessor.from_pretrained(model, trust_remote_code=True)

In [ ]:
def preprocess(example):
    img = example.get("image")

    messages = [{
        "role": "user",
        "content": f"<|image_1|>\n{example['llm_generated_idea']}"
    }, {
        "role": "assistant",
        "content": example.get("text", "Okay")
    }]

    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    inputs = processor(prompt, [img], return_tensors="pt")
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    inputs["labels"] = inputs["input_ids"].clone()

    return inputs


In [47]:
dataset = Dataset.from_parquet(filepath)
dataset = dataset.map(preprocess, remove_columns=["image", "llm_generated_idea", "text"])

Map:   3%|▎         | 68/2606 [00:02<01:43, 24.54 examples/s]

: 